<a href="https://colab.research.google.com/github/mikedornas/ibd/blob/main/ibd_tp_01_mike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Universidade Federal de Minas Gerais</center></h1>
<h3><center>Departamento de Ciência da Computação</center></h3>
<h3><center>Introdução a Banco de Dados -  Trabalho Prático #1</center></h3>


# 1. Introdução

### 1.1. Contexto

Nesse trabalho prático utilizaremos o banco de dados representado pelo schema abaixo:

![Modelo_BD.png](https://github.com/claudiovaliense/ibd/blob/main/data/schema.png?raw=true)

### 1.2. Entrega




1.   Você deve completar esse notebook, executar todas as células e submetê-lo no **Minha UFMG**.
2.   Você pode entregar até o dia 29/07/2021 às 23:59.
3. O trabalho é individual e vale 15 pontos.






# 2. Preparação do Ambiente

In [1]:
# importando os packages necessários
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# download do dump do banco de  dados:
!wget https://raw.githubusercontent.com/claudiovaliense/ibd/master/data/despesas_publicas_tp1.sql

--2021-07-29 19:17:23--  https://raw.githubusercontent.com/claudiovaliense/ibd/master/data/despesas_publicas_tp1.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277046 (271K) [text/plain]
Saving to: ‘despesas_publicas_tp1.sql’

despesas_publicas_t 100%[===================>] 270.55K  --.-KB/s    in 0.01s   

2021-07-29 19:17:23 (18.8 MB/s) - ‘despesas_publicas_tp1.sql’ saved [277046/277046]



### 2.1. Criação da conexão com o banco
* No momento da conexão, é informado o nome do arquivo onde será montado o banco de dados. No nosso caso, este arquivo não precisa ser pré-existente, porque iremos carregar as informações de um dump;
* Se este comando for executado com um arquivo que já existe e possui as mesmas tabelas que serão criadas, ocorrerá um erro no momento da execução do script mais à frente.

In [3]:
conn = sqlite3.connect('despesas.db')
cursor = conn.cursor()

### 2.2. Execução do script para inicializar e popular o banco de dados

* Criada a conexão, iremos utilizar o arquivo .sql disponibilizado para recuperar o schema e os dados do banco

In [4]:
f = io.open('despesas_publicas_tp1.sql', 'r', encoding='ISO-8859-1')
sql = f.read()
cursor.executescript(sql)

### 2.3. Realizando consultas no banco de dados

* Criada a conexão e realizada a carga dos dados no banco, pode ser executada qualquer consulta SQL;
* O SQLite3 permite que o retorno dos dados seja feito em vários formatos, entretanto, utilizaremos o retorno em um dataframe do Pandas.



**CONSULTA EXEMPLO:** todos os órgãos ordenados pela quantidade de orgãos subordinados de forma decrescente.

In [5]:
df = pd.read_sql_query("SELECT * FROM ORGAO ORDER BY QTD_ORGAOS_SUBORDINADOS DESC", conn)
df

,COD_ORGAO,NOME_ORGAO,QTD_ORGAOS_SUBORDINADOS
0,26000,Ministério da Educação,104
1,30000,Ministério da Justiça e Segurança Pública,6
2,22000,"Ministério da Agricultura, Pecuária e Abastec",5
3,24000,"Ministério da Ciência, Tecnologia, Inovações",4
4,52000,Ministério da Defesa,4
5,53000,Ministério da Integração Nacional,4
6,25000,Ministério da Fazenda,3
7,36000,Ministério da Saúde,3
8,20000,Presidência da República,2
9,39000,"Ministério dos Transportes, Portos e Aviação",2


#3. Consultas

Expresse cada consulta abaixo utilizando SQL: 


**1.** Retorne a quantidade de palavras em cada registro da coluna nome_orgao da tabela orgão.

In [50]:
df = pd.read_sql_query("SELECT NOME_ORGAO, (length(rtrim(NOME_ORGAO)) - length(replace(rtrim(NOME_ORGAO), ' ', '')) + 1) AS QTD_PALAVRAS FROM ORGAO", conn)
df

,NOME_ORGAO,QTD_PALAVRAS
0,Presidência da República,3
1,"Ministério da Agricultura, Pecuária e Abastec",6
2,"Ministério da Ciência, Tecnologia, Inovações",5
3,Ministério da Fazenda,3
4,Ministério da Educação,3
5,Ministério da Justiça e Segurança Pública,6
6,Ministério das Relações Exteriores,4
7,Ministério da Saúde,3
8,"Ministério dos Transportes, Portos e Aviação",6
9,Ministério do Meio Ambiente,4



**2.** Dado a coluna nome_orgao, retorne a primeira posição que a string 'da' aparece da tabela orgão 

In [25]:
df = pd.read_sql_query("SELECT * FROM ORGAO WHERE NOME_ORGAO LIKE '%da%' LIMIT 1", conn)
df

,COD_ORGAO,NOME_ORGAO,QTD_ORGAOS_SUBORDINADOS
0,20000,Presidência da República,2


**3.** Liste todos os nomes dos órgãos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [90]:
df = pd.read_sql_query("SELECT NOME_ORGAO, SUM(VALOR_DESPESA) AS VALOR_DESP_2018 FROM DESPESA_PUBLICA NATURAL JOIN ORGAO WHERE ANO_DESPESA='2018' GROUP BY NOME_ORGAO ORDER BY VALOR_DESP_2018 DESC", conn)
df

,NOME_ORGAO,VALOR_DESP_2018
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,"Ministério da Ciência, Tecnologia, Inovações",0.00
6,Ministério da Fazenda,0.00
7,Ministério da Integração Nacional,0.00
8,Ministério da Justiça e Segurança Pública,0.00
9,Ministério das Relações Exteriores,0.00


**4.** Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do órgão Ministério da Fazenda no primeiro semestre de 2018.




In [42]:
df = pd.read_sql_query("SELECT MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, NOME_PROGRAMA_GOVERNO, VALOR_DESPESA FROM (DESPESA_PUBLICA NATURAL JOIN ORGAO NATURAL JOIN PROGRAMA_GOVERNO) WHERE NOME_ORGAO='Ministério da Fazenda' AND ANO_DESPESA=2018 AND MES_DESPESA<7", conn)
df

,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,4,2018,Reserva de Contingência,GESTAO AEROPORTUARIA,0
1,1,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
2,2,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
3,3,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
4,4,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
5,5,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
6,6,2018,Reserva de Contingência,BRASIL SEM MISERIA,0


**5.** Liste o código e nome dos programas de governo que possuíram os 3 maiores valores de despesas totais na função de educação durante o ano de 2018.

In [49]:
df = pd.read_sql_query("SELECT COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO, SUM(VALOR_DESPESA) AS DESP_TOTAIS FROM (DESPESA_PUBLICA NATURAL JOIN FUNCAO NATURAL JOIN PROGRAMA_GOVERNO) WHERE NOME_FUNCAO='Educação' AND ANO_DESPESA='2018' GROUP BY COD_PROGRAMA_GOVERNO ORDER BY DESP_TOTAIS DESC LIMIT 3", conn)
df

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO,DESP_TOTAIS
0,24,BRASIL SEM MISERIA,387175.60
1,11,ENEM,334262.06
2,14,PRONATEC,60262.46


**6.** Transforme os caracteres em minúsculo e remova todos os termos 'da' na coluna nome_orgao da tabela orgão.

In [59]:
df = pd.read_sql_query("SELECT COD_ORGAO, lower(replace(NOME_ORGAO, ' da ', ' ')) AS NOME_ORGAO, QTD_ORGAOS_SUBORDINADOS FROM ORGAO", conn)
df

,COD_ORGAO,NOME_ORGAO,QTD_ORGAOS_SUBORDINADOS
0,20000,presidência república,2
1,22000,"ministério agricultura, pecuária e abastec",5
2,24000,"ministério ciência, tecnologia, inovações",4
3,25000,ministério fazenda,3
4,26000,ministério educação,104
5,30000,ministério justiça e segurança pública,6
6,35000,ministério das relações exteriores,1
7,36000,ministério saúde,3
8,39000,"ministério dos transportes, portos e aviação",2
9,44000,ministério do meio ambiente,2


**7.** Liste os órgãos governamentais e a quantidade de programas de governo distintos envolvidos nas despesas públicas desses órgãos. Nesta consulta, desconsidere valores de despesa negativos ou iguais a zero no período.

In [69]:
df = pd.read_sql_query("SELECT NOME_ORGAO, COUNT(DISTINCT NOME_PROGRAMA_GOVERNO) AS QTD_PROG FROM (DESPESA_PUBLICA NATURAL JOIN ORGAO NATURAL JOIN PROGRAMA_GOVERNO) WHERE VALOR_DESPESA>0 GROUP BY NOME_ORGAO", conn)
df

,NOME_ORGAO,QTD_PROG
0,"Ministério da Agricultura, Pecuária e Abastec",2
1,Ministério da Defesa,2
2,Ministério da Educação,4
3,Ministério da Saúde,2
4,Ministério do Desenvolvimento Social,1


**8.** Liste os nomes das subfunções das despesas públicas envolvidas para cada programa de governo. Sua consulta deve retornar o nome do programa de governo, a subfunção da despesa, a quantidade de entradas de despesas públicas e o valor total dessas despesas.

In [84]:
df = pd.read_sql_query("SELECT NOME_PROGRAMA_GOVERNO, NOME_SUBFUNCAO, COUNT(*) AS QTD_DESP, SUM(VALOR_DESPESA) AS VALOR_TOTAL_DESP FROM (DESPESA_PUBLICA NATURAL JOIN PROGRAMA_GOVERNO NATURAL JOIN DETALHAMENTO_FUNCAO NATURAL JOIN SUBFUNCAO) GROUP BY NOME_PROGRAMA_GOVERNO, NOME_SUBFUNCAO", conn)
df

,NOME_PROGRAMA_GOVERNO,NOME_SUBFUNCAO,QTD_DESP,VALOR_TOTAL_DESP
0,BRASIL SEM MISERIA,Alimentação e nutrição,112,100373.55
1,BRASIL SEM MISERIA,Assistência comunitária,117,227883.77
2,BRASIL SEM MISERIA,Comunicação social,1,0.00
3,BRASIL SEM MISERIA,Difusão do conhecimento científico e tecnológico,13,0.00
4,BRASIL SEM MISERIA,Educação infantil,2,0.00
...,...,...,...,...
60,VIVER SEM LIMITE,Desenvolvimento tecnológico e engenharia,11,0.00
61,VIVER SEM LIMITE,Educação básica,1,0.00
62,VIVER SEM LIMITE,Ensino superior,19,0.00
63,VIVER SEM LIMITE,Normatização e fiscalização,11,0.00


**9.** Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior número de órgãos subordinados.

In [87]:
df = pd.read_sql_query("SELECT COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA FROM DESPESA_PUBLICA WHERE COD_ORGAO IN (SELECT COD_ORGAO FROM ORGAO ORDER BY QTD_ORGAOS_SUBORDINADOS DESC LIMIT 1)", conn)
df

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,196,10,2018,Reserva de Contingência,0.0
1,197,1,2018,Reserva de Contingência,0.0
2,198,2,2018,Reserva de Contingência,0.0
3,199,3,2018,Reserva de Contingência,0.0
4,200,4,2018,Reserva de Contingência,0.0
...,...,...,...,...,...
289,485,8,2018,Reserva de Contingência,2400.0
290,486,9,2018,Reserva de Contingência,0.0
291,487,10,2018,Reserva de Contingência,0.0
292,488,11,2018,Reserva de Contingência,0.0


**10.** Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior valor total em despesas durante o ano de 2018.

In [95]:
df = pd.read_sql_query("SELECT COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA FROM DESPESA_PUBLICA WHERE COD_ORGAO IN (SELECT COD_ORGAO FROM DESPESA_PUBLICA NATURAL JOIN ORGAO WHERE ANO_DESPESA='2018' GROUP BY NOME_ORGAO ORDER BY SUM(VALOR_DESPESA) DESC LIMIT 1)", conn)
df

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,827,1,2018,Reserva de Contingência,0.0
1,828,2,2018,Reserva de Contingência,0.0
2,829,3,2018,Reserva de Contingência,0.0
3,830,4,2018,Reserva de Contingência,0.0
4,831,5,2018,Reserva de Contingência,0.0
5,832,6,2018,Reserva de Contingência,0.0
6,833,7,2018,Reserva de Contingência,0.0
7,834,8,2018,Reserva de Contingência,0.0
8,835,9,2018,Reserva de Contingência,334126.7
9,836,10,2018,Reserva de Contingência,0.0


**11.** Liste os códigos e nomes dos programas de governo relacionados ao órgão (ou órgãos, caso haja empate) que teve menos registros de despesas públicas durante o ano.

In [128]:
df = pd.read_sql_query("SELECT COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO FROM DESPESA_PUBLICA NATURAL JOIN PROGRAMA_GOVERNO WHERE COD_ORGAO IN (SELECT COD_ORGAO FROM (SELECT COD_ORGAO, NOME_ORGAO, COUNT(*) AS NUM_DESP FROM DESPESA_PUBLICA NATURAL JOIN ORGAO WHERE ANO_DESPESA='2018' GROUP BY COD_ORGAO ORDER BY COUNT(*) ASC) WHERE NUM_DESP IN (SELECT min(NUM_DESP) FROM (SELECT COD_ORGAO, NOME_ORGAO, COUNT(*) AS NUM_DESP FROM DESPESA_PUBLICA NATURAL JOIN ORGAO WHERE ANO_DESPESA='2018' GROUP BY COD_ORGAO ORDER BY COUNT(*) ASC)))", conn)
df

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,19,OLIMPIADAS
1,24,BRASIL SEM MISERIA


**12.**  Álgebra relacional: projeção do NOME_FUNCAO e NOME_SUBFUNCAO da junção natural das tabelas FUNCAO, DETALHAMENTO_FUNCAO e SUBFUNCAO

RESULT = 𝜋 NOME_FUNCAO, NOME_SUBFUNCAO (FUNCAO ⋈ DETALHAMENTO_FUNCAO ⋈ SUBFUNCAO)

In [132]:
df = pd.read_sql_query("SELECT NOME_FUNCAO, NOME_SUBFUNCAO FROM (FUNCAO NATURAL JOIN DETALHAMENTO_FUNCAO NATURAL JOIN SUBFUNCAO)", conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


**13.**  Álgebra relacional: projeção de MODALIDADE_DESPESA, VALOR_DESPESA e NOME_PROGRAMA_GOVERNO da junção natural das tabelas PROGRAMA_GOVERNO e DESPESA_PUBLICA onde a MODALIDADE_DESPESA = 'Reserva de Contingência'

RESULT = 𝜋 (MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO) 𝜎 MODALIDADE_DESPESA='Reserva de Contingência' (PROGRAMA_GOVERNO ⋈ DESPESA_PUBLICA)

In [133]:
df = pd.read_sql_query("SELECT MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO FROM (DESPESA_PUBLICA NATURAL JOIN PROGRAMA_GOVERNO) WHERE MODALIDADE_DESPESA='Reserva de Contingência'", conn)
df

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.0,BRASIL SEM MISERIA
1,Reserva de Contingência,0.0,BRASIL SEM MISERIA
2,Reserva de Contingência,0.0,BRASIL SEM MISERIA
3,Reserva de Contingência,0.0,BRASIL SEM MISERIA
4,Reserva de Contingência,0.0,BRASIL SEM MISERIA
...,...,...,...
671,Reserva de Contingência,0.0,BRASIL SEM MISERIA
672,Reserva de Contingência,0.0,BRASIL SEM MISERIA
673,Reserva de Contingência,0.0,BRASIL SEM MISERIA
674,Reserva de Contingência,0.0,BRASIL SEM MISERIA


**14.**  Álgebra relacional: projeção de VALOR_DESPESA e NOME_ORGAO da junção natural das tabelas DESPESA_PUBLICA e ORGAO onde o NOME_ORGAO = 'Ministério da Fazenda'

RESULT = 𝜋 (VALOR_DESPESA, NOME_ORGAO) 𝜎 NOME_ORGAO='Ministério da Fazenda' (DESPESA_PUBLICA ⋈ ORGAO)

In [134]:
df = pd.read_sql_query("SELECT  VALOR_DESPESA, NOME_ORGAO FROM (DESPESA_PUBLICA NATURAL JOIN ORGAO) WHERE NOME_ORGAO='Ministério da Fazenda'", conn)
df

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda
1,0,Ministério da Fazenda
2,0,Ministério da Fazenda
3,0,Ministério da Fazenda
4,0,Ministério da Fazenda
5,0,Ministério da Fazenda
6,0,Ministério da Fazenda
7,0,Ministério da Fazenda
8,0,Ministério da Fazenda
9,0,Ministério da Fazenda


**15.**  Álgebra relacional: projeção de NOME_PROGRAMA_GOVERNO e VALOR_DESPESA da junção natural das tabelas DESPESA_PUBLICA, PROGRAMA_GOVERNO e ORGAO onde o NOME_ORGAO = 'Ministério da Saúde'

RESULT = 𝜋 (NOME_PROGRAMA_GOVERNO, VALOR_DESPESA) 𝜎 NOME_ORGAO='Ministério da Saúde' (DESPESA_PUBLICA ⋈ PROGRAMA_GOVERNO ⋈ ORGAO)

In [135]:
df = pd.read_sql_query("SELECT  NOME_PROGRAMA_GOVERNO, VALOR_DESPESA FROM (DESPESA_PUBLICA NATURAL JOIN PROGRAMA_GOVERNO NATURAL JOIN ORGAO) WHERE  NOME_ORGAO='Ministério da Saúde'", conn)
df

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.0
1,VIVER SEM LIMITE,0.0
2,VIVER SEM LIMITE,0.0
3,VIVER SEM LIMITE,0.0
4,VIVER SEM LIMITE,0.0
...,...,...
214,MAIS MEDICOS,0.0
215,MAIS MEDICOS,0.0
216,MAIS MEDICOS,0.0
217,MAIS MEDICOS,0.0


# 4. Fechamento da conexão com o banco

* Após serem realizadas todas as consultas necessárias, é aconselhavel encerrar formalmente a conexão com o banco de dados

In [ ]:
conn.close()